In [ ]:
from glob import glob
import  pandas as pd
import soundfile as sf
import os

df_trimmed = pd.read_csv("trimmed_durations_de.csv")

In [ ]:
csv_files = glob('*_German*/*.csv')
csv_files = sorted(csv_files)


df_aggregated = pd.DataFrame(columns=["file", "duration_before_matching","duration_after_mathing", "duration_after_trimming", "total_files", "assigned", "not_assigned"])
dfs = []
for csv_file in csv_files:
    print(csv_file)
    df = pd.read_csv(csv_file)
    df["duration"] = df["end"] - df["start"]

    df["filename"] = df["sentenceNumber"].apply(lambda x: "DE" + str(x).zfill(8) + ".wav")
    # merge with trimmed durations and df_all using filename as key and keep the df_all index
    df = df[['status', 'filename', 'sentenceNumber', 'sentence', 'asr']].merge(df_trimmed[["deliverable","filename" ,"duration_before_trimming", "trim_start",	"trim_end",	"duration_after_trimming"]], on="filename", how="left")

    wav_file  = csv_file[:-4]
    data, samplerate = sf.read(wav_file)
    duration = len(data) / samplerate
    row_aggregated = {"file": os.path.basename(wav_file), "duration_before_matching": duration}

    row_aggregated["duration_after_mathing"] = df["duration_before_trimming"].sum()
    row_aggregated["duration_after_trimming"] = df["duration_after_trimming"].sum()
    row_aggregated["total_files"] = len(df)
    row_aggregated["assigned"] = df[df['status'] == "assigned"].shape[0]
    row_aggregated["not_assigned"] = df[df['status'] == "not_assigned"].shape[0]
    
    df_aggregated = df_aggregated.append(row_aggregated, ignore_index=True)
    dfs.append(df)


In [ ]:
df_all = pd.concat(dfs)
df_all.to_csv("all_samples_provided-de.csv", index=False)

In [ ]:
df_aggregated.to_csv("aggregated-de.csv", index=False)

In [ ]:
df_all

In [ ]:
df_aggregated

In [ ]:
# Take 20 secs of wav_file and save it as wav_file_20s.wav

!ffmpeg -i 'batches_GermanM1/DE00005198 - DE00006004.wav' -ss 00:00:00 -t 00:01:00 -c copy German_60s.wav